## multithreading with a log queue and button controls

In [3]:
from nbmultitask import ThreadWithLogAndControls

In [4]:
from time import sleep

# the target function will be passed a Queue object via a keyword argument `log`
def fn(log):
    i = 1
    # be careful with loops... (in order for the stop button to work)
    while i <= 5:
        log.put('%i...' % i)
        sleep(1.5)
        i+=1

task = ThreadWithLogAndControls(target=fn, name="do some stuff")

In [5]:
task.control_panel()

## looping, working with a shared variable

In [9]:
from nbmultitask import ThreadWithLogAndControls

In [10]:
from time import sleep

# this function will be looped when we pass `loop=True` to the constructor below
def fn(x, log):
    log.put(var['value'])
    x['value'] = x['value']**1.01
    sleep(0.1)

x = {'value': 2} # using shared memory
task = ThreadWithLogAndControls(target=fn, args=(x,), loop=True, name="exponential growth")

In [11]:
task.control_panel()

## multiprocessing

In [13]:
from nbmultitask import ProcessWithLogAndControls

In [14]:
from time import sleep

def fn(x, log):
    while True:
        log.put(x.value)
        x.value = x.value**1.01
        sleep(0.1)

from multiprocessing import Value
x = Value('f',2)
task = ProcessWithLogAndControls(target=fn, args=(x,), name="exponential growth")

In [15]:
task.control_panel()

## subclassing `multiprocessing.Process` with a `work` function

In [20]:
from nbmultitask import withLogAndControls
from threading import Thread
from multiprocessing import Event, Queue, Process
from ipywidgets import Button, Layout
from IPython.display import display
from time import sleep
import random

class Snoozer(Process):
    def __init__(self, *args, **kwargs):
        super().__init__(*args,**kwargs)
        # add state and buttons
        self.snoozing = Event()
        self.snooze_button = Button(description='SNOOZE...')
        self.snooze_button.layout = layout=Layout(width='50%', height='80px')
        self.snooze_button.style.button_color = 'darkslateblue'
        self.snooze_button.style.font_weight = 'bold'
        self.snooze_button.button_style = 'primary'
        self.snooze_button.on_click(lambda evt: self.snoozing.set())
        
    def show_snooze_button(self):
        display(self.snooze_button)
    
    # this function will be called as though it were passed via `target=`,
    # except that it will also have access to the Process Snoozer instance
    # (i.e. the `self.snoozing` Event)
    def work(self, log):
        if self.snoozing.is_set():
            snooze_time = random.random()*5
            log.put('Snoozing for %s seconds!' % snooze_time)
            sleep(snooze_time)
            self.snoozing.clear()
            
        sleep(0.01) # no need to spin the wheels


In [21]:
# `withLogAndControls` can be passed any subclass of `multiprocessing.Process` or `threading.Thread`
snoozer = withLogAndControls(Snoozer)(loop=True)
snoozer.control_panel()

In [22]:
snoozer.show_snooze_button()

## starting/stopping a Flask server

In [25]:
import flask
from nbmultitask import ProcessWithLogAndControls

class Server(ProcessWithLogAndControls):
    def __init__(self,app,*args,**kwargs):
        self.app = app
        ProcessWithLogAndControls.__init__(self,*args,**kwargs)

    def work(self,log):
        self.log.put('starting server')
#         with self.output:
        self.app.run()

app = flask.Flask('myapp')

server = Server(app)
server.control_panel()